## Documentation

- API Guide can be found in this link https://ib-insync.readthedocs.io/api.html
- Options Example (SPX) with ib_insync https://nbviewer.org/github/erdewit/ib_insync/blob/master/notebooks/option_chain.ipynb


In [64]:
from ib_insync import *
import pandas as pd
util.startLoop()
from datetime import date
from datetime import datetime

def verify_daycount(dates):
    ds = []
    today = date.today()
    today = datetime.strftime(today, "%Y%m%d")
    today = datetime.strptime(today, "%Y%m%d")
    for adate in dates:
        date_obj = datetime.strptime(adate, "%Y%m%d")
        days_diff = (date_obj - today).days
        if (40 < days_diff < 50) or (90 < days_diff < 150):
            ds.append(adate)
    return ds
    

ib = IB()
ib.connect('127.0.0.1', 7496, clientId = 3)

tick = 'AAPL'
print( " >>> DEFINING STOCK ")
stk = Stock(tick, 'SMART', 'USD')
ib.qualifyContracts(stk)
ib.reqMarketDataType(4) #wait a bit
chains = ib.reqSecDefOptParams(stk.symbol, '', stk.secType, stk.conId)
ds_chains = pd.DataFrame(chains)
expirations = sorted(ds_chains['expirations'].iloc[0])
expirations = verify_daycount(expirations) #convert expiration day to number of days left to expire
strikes = ds_chains['strikes'].iloc[0]
contractID = ds_chains['underlyingConId'].iloc[0]
rights = ['P', 'C'] #easier for algirthm built # P = Put ; C = Call

#organize all Option Contracts into data set
print( " >>> DEFINING OPTIONS ")
contracts = [Option(tick, ex, s, r, 'SMART') for r in rights for ex in expirations for s in strikes]
print( " >>> QUALIFYING OPTIONS ")
qualifiedContracts = ib.qualifyContracts(*contracts)
print( " >>> RETRIEVING DETAILS ")
qualifiedContractDetails = ib.reqTickers(*qualifiedContracts)
ds = pd.DataFrame()
print ( " >>> GETTING GREEKS AND OPTIONS INFO")
for qcd in qualifiedContractDetails:
    try:
        details = qcd.contract
        exp, strike, right = details.lastTradeDateOrContractMonth, details.strike, details.right
        greek = qcd.modelGreeks
        delta, theta, gamma, vega, pvDiv, optPrice, impliedVol = greek.delta, greek.theta, greek.gamma, greek.vega, greek.pvDividend, greek.optPrice, greek.impliedVol
        row = {
            'Symbol': tick, 'Expiration': exp , 'Strike': strike, 'Right': right,
            'Delta': delta, 'Theta': theta, 'Vega': vega, 'Gamma': gamma,
            'impVol': impliedVol, 'optPrice': optPrice}
        ds = pd.concat([ds, pd.DataFrame([row])], axis = 0, ignore_index = True)
    except:
        print ( " >>> NOGOOD CONTRACT")

#next: call all options during market time to get greek values
def days_diff(adate):
    today = date.today()
    today = datetime.strftime(today, "%Y%m%d")
    today = datetime.strptime(today, "%Y%m%d")
    date_obj = datetime.strptime(adate, "%Y%m%d")
    return (date_obj - today).days

ds['daysdiff'] = list(map(days_diff, ds['Expiration'])) 
   
ds['Delta'] = abs(ds['Delta'])
ds['Theta'] = abs(ds['Theta'])
# ds_p1 = ds[(0.1 <= ds['Delta'] <= 0.2) & (ds['Right'] == 'P') & (days_diff(ds['Expiration'], 90, 150))]
ds_p1 = ds[(ds['Right'] == 'P') & (ds['daysdiff'] >= 90) & (ds['daysdiff'] <= 150) & (ds['Delta'] >= 0.1) & (ds['Delta'] <= 0.2)]
ds_p1['REGEL'] = ['P1'] * len(ds_p1)
ds_p2 = ds[(ds['Right'] == 'P') & (ds['daysdiff'] >= 40) & (ds['daysdiff'] <= 50) & (ds['Delta'] >= 0.2) & (ds['Delta'] <= 0.3)]
ds_p2['REGEL'] = ['P2'] * len(ds_p2)
ds_c3 = ds[(ds['Right'] == 'C') & (ds['daysdiff'] >= 40) & (ds['daysdiff'] <= 50) & (ds['Delta'] >= 0.2) & (ds['Delta'] <= 0.3)]
ds_c3['REGEL'] = ['C3'] * len(ds_c3)
ds_c4 = ds[(ds['Right'] == 'C') & (ds['daysdiff'] >= 90) & (ds['daysdiff'] <= 150) & (ds['Delta'] >= 0.1) & (ds['Delta'] <= 0.2)]
ds_c4['REGEL'] = ['C4'] * len(ds_c4)

main = pd.concat([ds_p1, ds_p2, ds_c3, ds_c4], ignore_index=True)
def p_marg(p1, p2):
    if (p1 == p2 - 10) | (p1 == p2 - 20):
        return True
    return False

def c_marg(c3, c4):
    if (c4 == c3 + 10) | (c4 == c3 + 20):
        return True
    return False

def p_theta(p1, p2):
    if (p2 > p1 * 1.5):
        return True
    return False

def c_theta(c3, c4):
    if (c3 > c4 * 1.5):
        return True
    return False

def bundle(ds):
    p1 = ds[ds['REGEL'] == 'P1']
    p2 = ds[ds['REGEL'] == 'P2']
    c3 = ds[ds['REGEL'] == 'C3']
    c4 = ds[ds['REGEL'] == 'C4']
    ps = []
    cs = []
    i = 0

    #convert two loops to function prperly (rearrange functions in sheet)
    for i in range(len(p1)):
        j = 0
        for j in range(len(p2)):
            # code begin
            if p_marg(p1.iloc[i]['Strike'], p2.iloc[j]['Strike']) & p_theta(p1.iloc[i]['Theta'], p2.iloc[j]['Theta']):
                ps.append((i, j))
            # code end
            j += 1
        i += 1

    for i in range(len(c3)):
        j = 0
        for j in range(len(c4)):
            # code begin
            if c_marg(c3.iloc[i]['Strike'], c4.iloc[j]['Strike']) & c_theta(c3.iloc[i]['Theta'], c4.iloc[j]['Theta']):
                cs.append((i, j))
            # code end
            j += 1
        i += 1
    

    legs_indx = [(p1, p2, c3, c4) for (p1, p2) in ps for (c3, c4) in cs]

    dss = []
    for (a, b, c, d) in legs_indx:
        ds = pd.concat([p1.iloc[a].to_frame().T, p2.iloc[b].to_frame().T, c3.iloc[c].to_frame().T, c4.iloc[d].to_frame().T], ignore_index=True)
        # df = pd.DataFrame.from_records(ds)
        dss.append(ds)
    
    return dss


bundles = bundle(main)
pd.concat(bundles).to_csv('options_03.csv')
ib.disconnect()


 >>> DEFINING STOCK 
 >>> DEFINING OPTIONS 
 >>> QUALIFYING OPTIONS 


Error 200, reqId 5: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20220826', strike=30.0, right='P', exchange='SMART')
Error 200, reqId 8: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20220826', strike=41.25, right='P', exchange='SMART')
Error 200, reqId 6: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20220826', strike=35.0, right='P', exchange='SMART')
Error 200, reqId 7: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20220826', strike=40.0, right='P', exchange='SMART')
Error 200, reqId 9: No security definition has been found for the request, contract: Option(symbol='AAPL', lastTradeDateOrContractMonth='20220826', strike=42.5, right='P', exchange='SMART')
Error 200, reqId 10: No security definition has b

 >>> RETRIEVING DETAILS 
 >>> GETTING GREEKS AND OPTIONS INFO
 >>> NOGOOD CONTRACT
 >>> NOGOOD CONTRACT
 >>> NOGOOD CONTRACT
 >>> NOGOOD CONTRACT
 >>> NOGOOD CONTRACT
 >>> NOGOOD CONTRACT
 >>> NOGOOD CONTRACT
 >>> NOGOOD CONTRACT
 >>> NOGOOD CONTRACT
 >>> NOGOOD CONTRACT
 >>> NOGOOD CONTRACT
 >>> NOGOOD CONTRACT
 >>> NOGOOD CONTRACT
 >>> NOGOOD CONTRACT
 >>> NOGOOD CONTRACT
 >>> NOGOOD CONTRACT
 >>> NOGOOD CONTRACT
 >>> NOGOOD CONTRACT
 >>> NOGOOD CONTRACT
 >>> NOGOOD CONTRACT
 >>> NOGOOD CONTRACT
 >>> NOGOOD CONTRACT
 >>> NOGOOD CONTRACT
 >>> NOGOOD CONTRACT
 >>> NOGOOD CONTRACT
 >>> NOGOOD CONTRACT
 >>> NOGOOD CONTRACT
 >>> NOGOOD CONTRACT
 >>> NOGOOD CONTRACT
 >>> NOGOOD CONTRACT
 >>> NOGOOD CONTRACT
 >>> NOGOOD CONTRACT
 >>> NOGOOD CONTRACT
 >>> NOGOOD CONTRACT
 >>> NOGOOD CONTRACT
 >>> NOGOOD CONTRACT
 >>> NOGOOD CONTRACT
 >>> NOGOOD CONTRACT
 >>> NOGOOD CONTRACT
 >>> NOGOOD CONTRACT
 >>> NOGOOD CONTRACT
 >>> NOGOOD CONTRACT
 >>> NOGOOD CONTRACT
 >>> NOGOOD CONTRACT
 >>> NOGOOD CO

KeyError: 'Expiration'

In [4]:
ib.disconnect()